[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/huggingface/education-toolkit/blob/main/02_ml-demos-with-gradio.ipynb)



💡 **Bem-vindo!**

Nós reunimos um conjunto de ferramentas que instrutores universitários e organizadores podem usar para preparar laboratórios, tarefas ou aulas.
O conteudo foi projetado de uma forma autocontida, para ser facilmente incorporado no currículo existente. O conteúdo é gratuito e usa
tecnologias amplamente reconhecidas como Open Source (`transformers`, `gradio`, etc).

Alternativamente, você pode pedir para que alguém no time da Hugging Face rodar os tutoriais para suas aulas via a iniciativa [Tour de demo.cratização de ML](https://huggingface2.notion.site/ML-Demo-cratization-tour-with-66847a294abd4e9785e85663f5239652)!

Você também pode encontrar todos os tutoriais e recursos que nós montamos [aqui](https://huggingface2.notion.site/Education-Toolkit-7b4a9a9d65ee4a6eb16178ec2a4f3599).

# Tutorial: Construa e Hospede uma demonstração de Aprendizado de Máquina com o Gradio ⚡ & Hugging Face 🤗

**Objetivos de aprendizado:** Os objetivos deste tutorial são aprender como

1. Construir uma demonstração rápida para o seu modelo de aprendizado de máquina em Python usando a biblioteca `gradio`
2. Hospedar a demo de graça com o Hugging Face Spaces
3. Adicionar sua demo à Hugging Face org para sua aula ou conferência. Isso incluirá:
  * Uma configuração passo a passo para instrutores (ou organizadores de conferências)
  * Envio de instruções para estudantes (ou participantes de conferências)

**Duração**: 20 a 40
 minutos

**Pré-requisitos:** Conhecimento de Python e familiaridade básica com aprendizado de máquina

**Autor**: [Abubakar Abid](https://twitter.com/abidlabs) (feel free to ping me with any questions about this tutorial)
**Tradutor**: [Rafael Zimmer](https://github.com/rzimmerdev)

Todas as etapas podem ser realizadas de graça! Tudo que você irá precisar é uma conexão à internet e um lugar onde possa programar em Python 👩‍💻

## Porque ?

**Demos** de modelos de aprendizado de máquina são uma parte importante de _cursos_ e _conferências_ sobre aprendizado de máquina. Demonstrações permitem:

* que desenvolvedores de modelos **apresentem** os seus trabalhos a uma ampla audiência
* um aumento na **reprodutividade** da pesquisa sobre aprendizado de máquina
* que usuários *identifiquem e debugem* pontos de falhas de modelos mais facilmente


Para um exemplo rápido sobre o que gostariamos de montar, confira o [Keras Org na Hugging Face](https://huggingface.co/keras-io), que inclui uma descrição
e uma coleção de Modelos e Espaços construídos pela comunidade do Keras. Qualquer Espaço pode ser aberto em seu navegador, e você poderá usar o modelo imediatamente, como mostrado a seguir:

![](https://i.ibb.co/7y6DGjB/ezgif-5-cc52b7e590.gif)




## 1. Montando demonstrações rápidas em Python de Aprendizado de Máquina usando o Gradio

`gradio` é uma biblioteca de Python extremamente útil que permite a construção de demonstrações online simplesmente especificando uma lista de componentes de entrada e saída esperados pelo seu modelo de aprendizado de máquina.

O que podem então ser considerados como componentes de entrada e saída? O Gradio vem com um conjunto de componentes pré-definidos para diversos tipos de modelos de aprendizado de máquina. A seguir temos alguns exemplos:

* Para um **classificador de imagem**, a entrada esperada é do tipo `Imagem` e a saída do tipo `Label`.
* Para um **modelo reconhecedor de fala**, o componente de entrada é do tipo `Microphone` (que permite aos usuários gravar áudio pelo navegador), ou áudio (que permite usuários puxar e soltar arquivos de áudio), enquanto a saída é do tipo `Text`.
* Para um **modelo de questões e respostas**, **2 entradas** são esperadas: [`Text`, `Text`], uma para a caixa de texto com um parágrafo, e outro para questão, e a saída é única, do tipo `Text` correspondendo à resposta.

Você entendeu a idéia... (para todos os componentes aceitos, [acesse a documentação](https://gradio.app/docs/))

Além da entrada e saída, o Gradio espera também um terceiro parâmetro, que é a predição do modelo em si. Esse parâmetro pode ser ***qualquer* função regular do Python** que receba parâmetro(s) correspondendo ao(s) componente(s) de entrada e que tenha como retorno valor(es), correspondendo ao(s) componente(s) de saída.

Chega de discutir. Vamos ao programa!

In [2]:
# Primeiro, installe o gradio
!pip install --quiet gradio

In [3]:
import numpy as np

def sepia(image):
    sepia_filter = np.array(
        [[0.393, 0.769, 0.189], 
         [0.349, 0.686, 0.168], 
         [0.272, 0.534, 0.131]]
    )
    sepia_img = image.dot(sepia_filter.T)
    sepia_img /= sepia_img.max()
    return sepia_img

In [4]:
import gradio as gr

# Escreva uma simples linha para criar uma Interface Gráfica
gr.Interface(fn=sepia, inputs="image", outputs="image").launch(share=True);

Running on local URL:  http://127.0.0.1:7860/
Running on public URL: https://10801.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


O código acima deverá produzir uma simples interface gráfica dentro do ‘notebook’, que lhe permitirá enviar uma entrada e ver a saída como retorno da sua função.

Note também que definimos a `Interface` usando os três ingredientes mencionados anteriormente:
* Uma função
* Componente(s) de entrada
* Componente(s) de saída

Fizemos um exemplo simples para imagens, mas a idéia fundamental vale para quaisquer outros tipos de dados. Por exemplo, abaixo há uma interface que irá
gerar um tom musical quando receber alguns parâmetros diferentes (o código específico dentro de `generate_tone()` não é importante para os propósitos deste tutorial):

In [ ]:
import numpy as np
import gradio as gr

def generate_tone(note, octave, duration):
    sampling_rate = 48000
    a4_freq, tones_from_a4 = 440, 12 * (octave - 4) + (note - 9)
    frequency = a4_freq * 2 ** (tones_from_a4 / 12)
    audio = np.linspace(0, int(duration), int(duration) * sampling_rate)
    audio = (20000 * np.sin(audio * (2 * np.pi * frequency))).astype(np.int16)
    return sampling_rate, audio

gr.Interface(
    generate_tone,
    [
        gr.inputs.Dropdown(["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B"], type="index"),
        gr.inputs.Slider(4, 6, step=1),
        gr.inputs.Textbox(type="number", default=1, label="Duration in seconds"),
    ],
    "audio",
    title="Generate a Musical Tone!"
).launch(share=True)

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://20619.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


(<fastapi.applications.FastAPI at 0x7f84a8c6f850>,
 'http://127.0.0.1:7860/',
 'https://20619.gradio.app')

**Desafio #1**: construa uma demonstração do Gradio que receba uma imagem e retorne a mesma image *virada de cabeça pra baixo* em menos de 10 linhas de código em Python.

Há diversos outros exemplos para você testar na [página de introdução](https://gradio.app/getting_started/) do Gradio, que cobre funcionalidades adicionais, como:
* Adicionar exemplos para entradas
* Adicionar _estados_ (para chatbots, por exemplo)
* Comparar demonstrações facilmente usando o parâmetro `share` (<-- bem interessante 😎)

É especialmente fácil transformar um modelo `transformers` do Hugging Face's Model Hub em uma demo, usando o método especial `gr.Interface.load`

Testaremos um modelo de texto para fala, construído pelo Facebook:

In [ ]:
import gradio as gr

gr.Interface.load("huggingface/facebook/fastspeech2-en-ljspeech").launch(share=True);

Aqui é carregado o código para a demo do [GPT-J](https://huggingface.co/EleutherAI/gpt-j-6B), um grande modelo de linguagem & algumas entradas como exemplo:

In [ ]:
import gradio as gr

examples = [["The Moon's orbit around Earth has"], ["There once was a pineapple"]]

gr.Interface.load("huggingface/EleutherAI/gpt-j-6B", examples=examples).launch(share=True);

Fetching model from: https://huggingface.co/EleutherAI/gpt-j-6B
Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://30262.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


**Desafio #2**: vá para o [Model Hub da Hugging Face](https://huggingface.co/models) e escolha um modelo que realize alguma das outras tarefas aceitas pela biblioteca de `transformers` (diferente dos dois que você acabou de ver: geração de texto e texto para fala). Crie uma demonstração do Gradio para o modelo usando o `gr.Interface.load`.

## 3. Hospede a Demo (de graça) no Hugging Face Spaces

Quando tiver feito a demonstração com o Gradio, poderá hospedá-la permanentemente no Hugging Face Spaces facilmente:

A seguir estão os passos a seguir (demonstrados no GIF abaixo):

A. Primeiro, crie uma conta na Hugging Face se já não tiver uma; visite https://huggingface.co/ and clicking e clique em "Sign Up"

B. Após entrar, clique na sua foto de perfil e depois clique em "Novo Espaço", logo abaixo para chegar nessa página: https://huggingface.co/new-space

C. Dê um nome ao seu espaço e uma licença. Selecione "Gradio" como o SDK do Space, e selecione "Público" se decidir dar acesso ao seu Space para todos e o código dentro do mesmo

D. No mesmo ambiente, há uma página com instruções de como carregar arquvios do seu repositório Git para o Space. Será necessário
adicionar um arquiv `requirements.txt` para especificar quaisquer dependências de pacotes do Python.

E. Após enviar seus arquivos, sente-se e relaxe! Os Spaces irão automaticamente construir a sua demonstração do Gradio, permitindo que você as compartilhe com qualquer um, em qualquer lugar!

![GIF](https://huggingface.co/blog/assets/28_gradio-spaces/spaces-demo-finalized.gif)





Você pode até embutir a sua demonstração do Gradio em qualquer página — seja num blog, em um portfólio online, ou até mesmo em um Notebook do Colab, como foi feito a seguir com um modelo de reconhecimento de retratos do Pictionary:

In [ ]:
from IPython.display import IFrame
IFrame(src='https://hf.space/gradioiframe/abidlabs/Draw/+', width=1000, height=800)

**Desafio #3**: Carregue a sua demonstração do Gradio para o Hugging Face Spaces e receba um link permanente para o mesmo. Divulgue o link permanente com alguém (um colega, colaborador, amigo, usuário, etc.) - e receba opiniões sobre o seu modelo de aprendizado de máquina.

## 3. Adicione sua demo a Hugging Face org para sua aula ou conferência

#### **Setup** (para instrutores ou organizadores de conferências)

A. Primeiro, crie uma conta na Hugging Face se já não tiver uma; visite https://huggingface.co/ e clique em "Sign Up"

B. Após entrar, clique na sua foto de perfil e depois clique em "Nova Organização" logo abaixo para acessar essa página: https://huggingface.co/organizations/new

C. Preencha a informação relativa a sua aula ou conferência. Recomendamos criar uma organização separada toda vez que um curso diferente for dado (por exemplo, "Stanford-CS236g-2022") ou para cada ano em que houver a conferência.

D. Sua organização será criada, e agora novos usuários poderão enviar pedidos de inscrição ao visitar a página da sua organização.

E. Opcionalmente, você também pode mudar as configurações ao clicar em "Organization settings". Tipicamente, para aulas e conferências, deverá ir em `Settings > Members` e mudar a "Default role for new members" (cargo padrão para novos membros) para "write" (escrever), o que permitirá novos membros enviar Spaces, mas não mudar as configurações da organização.

#### Para estudantes ou participantes de conferências

A. Peça a seu instrutor / organizador da conferência pelo link da página da Organização, se já não o tiver.

B. Visite a página da Organização e clique em "Request to join this org" (Pedir para se juntar à essa organização), se já não fizer parte da mesma.

C. Finalmente, após ter sido aprovado para entrar (e já ter construído sua demonstração do Gradio e enviado para o Spaces - retorne à Seção 1 e 2 para ver como), simplesmente vá ao seu Space e acesse `Settings > Rename or transfer this space` (renomear ou transferir este Space) e selecione a organização desejada no menu `New owner`. Clique no botão para confirmar e o seu Space já será adicionado ao Space do seu curso ou conferência.